<a href="https://colab.research.google.com/github/thwlswkdgh/GAN/blob/main/code_practice/3D_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#encoder

from keras.datasets import mnist
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model

# Download the data and format for learning
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# How much encoding do we want for our setup?
encoding_dimension = 256  

# Keras has an input shape of 784
input_layer = Input(shape=(784,))
encoded_layer = Dense(encoding_dimension, activation='relu')(input_layer)
decoded = Dense(784, activation='sigmoid')(encoded_layer)

# Build the Model
ac = Model(input_layer, decoded)

# Create an encoder model that we will save later
encoder = Model(input_layer, encoded_layer)

# Train the autoencoder model, ac
ac.compile(optimizer='adadelta', loss='binary_crossentropy')
ac.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Save the Predicted Data x_train
x_train_encoded = encoder.predict(x_train)
np.save('x_train_encoded.npy',x_train_encoded)
# Save the Predicted Data x_test
x_test_encoded = encoder.predict(x_test)
np.save('x_test_encoded.npy',x_test_encoded)
# Save the Encoder model
encoder.save('encoder_model.h5')

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/100
235/235 [==============================] - 11s 42ms/step - loss: 0.6951 - val_loss: 0.6946
Epoch 2/100
235/235 [==============================] - 9s 39ms/step - loss: 0.6940 - val_loss: 0.6935
Epoch 3/100
235/235 [==============================] - 6s 24ms/step - loss: 0.6930 - val_loss: 0.6925
Epoch 4/100
235/235 [==============================] - 5s 22ms/step - loss: 0.6920 - val_loss: 0.6915
Epoch 5/100
235/235 [==============================] - 5s 22ms/step - loss: 0.6910 - val_loss: 0.6905
Epoch 6/100
235/235 [==============================] - 5s 22ms/step - loss: 0.6900 - val_loss: 0.6895
Epoch 7/100
235/235 [==============================] - 5s 22ms/step - loss: 0.6890 - val_loss: 0.6885
Epoch 8/100
235/235 [==============================] - 5s 22ms/step - loss: 0.6881 - val_loss: 0.6875
Epoch 9/100
235/235 [==============================] - 5s 22ms/step - loss: 0.6871 - val_loss: 0.6865
Epoch 10/100
235

In [ ]:
!pip install -q kaggle
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d daavoo/3d-mnist

 98% 149M/153M [00:02<00:00, 67.6MB/s]
100% 153M/153M [00:02<00:00, 65.8MB/s]


In [ ]:
!unzip 3d-mnist -d 3d-mnist

Archive:  3d-mnist.zip
  inflating: 3d-mnist/full_dataset_vectors.h5  
  inflating: 3d-mnist/plot3D.py      
  inflating: 3d-mnist/test_point_clouds.h5  
  inflating: 3d-mnist/train_point_clouds.h5  
  inflating: 3d-mnist/voxelgrid.py   


In [ ]:
#discriminator

#!/usr/bin/env python3
import sys
import numpy as np
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv3D, Deconv3D
from keras.layers.core import Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model


class Discriminator(object):
    def __init__(self, side=16):
        self.INPUT_SHAPE = (side,side,side,3)
        self.OPTIMIZER = Adam(lr=0.000001, beta_1=0.5)


        self.Discriminator = self.model()
        self.Discriminator.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER, metrics=['accuracy'] )
        # self.save_model()
        self.summary()

    def block(self,first_layer,filter_size=512,kernel_size=(3,3,3)):

        x = Conv3D(filters=filter_size, kernel_size=kernel_size, kernel_initializer='glorot_normal',
                    bias_initializer='zeros', padding='same')(first_layer)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        return x

    def model(self):
        input_layer = Input(shape=self.INPUT_SHAPE)
        x = self.block(input_layer,filter_size=8)
        x = self.block(x,filter_size=16,)
        x = self.block(x,filter_size=32)
        x = self.block(x,filter_size=64)


        x = Conv3D(filters=1, kernel_size=(3,3,3),
                    strides=(1,1,1), kernel_initializer='glorot_normal',
                    bias_initializer='zeros', padding='valid')(x)
        x = BatchNormalization()(x)
        x = Flatten()(x)
        output_layer = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=input_layer, outputs=output_layer)

        return model

    def summary(self):
        return self.Discriminator.summary()

    def save_model(self):
        plot_model(self.Discriminator.model, to_file='Discriminator_Model.png')



In [ ]:
# generator

#!/usr/bin/env python3
import sys
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv3D, Deconv3D
from keras.layers import Input, BatchNormalization, Dense, Reshape
from keras.layers.core import Activation
from tensorflow.keras.optimizers import Adam, SGD
# from keras.utils import plot_model

class Generator(object):
    def __init__(self, latent_size=100):

        self.INPUT_SHAPE = (1, 1, 1, latent_size)
        # self.OPTIMIZER = Adam(lr=0.0001,beta_1=0.5)
        self.OPTIMIZER = SGD(lr=0.001, nesterov=True)


        self.Generator = self.model()
        self.Generator.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)
        # self.save_model()
        self.summary()

    def block(self,first_layer,filter_size=512,stride_size=(2,2,2),kernel_size=(4,4,4),padding='same'):

        x = Deconv3D(filters=filter_size, kernel_size=kernel_size,
                    strides=stride_size, kernel_initializer='glorot_normal',
                    bias_initializer='zeros', padding=padding)(first_layer)
        x = BatchNormalization()(x)
        x = Activation(activation='relu')(x)

        return x


    def model(self):
        input_layer = Input(shape=self.INPUT_SHAPE)

        x = self.block(input_layer,filter_size=256,stride_size=(1,1,1),kernel_size=(4,4,4),padding='valid')
        x = self.block(x,filter_size=128,stride_size=(2,2,2),kernel_size=(4,4,4))

        x = Deconv3D(filters=3, kernel_size=(4,4,4),
                    strides=(2,2,2), kernel_initializer='glorot_normal',
                    bias_initializer='zeros', padding='same')(x)
        x = BatchNormalization()(x)
        output_layer = Activation(activation='sigmoid')(x)


        model = Model(inputs=input_layer, outputs=output_layer)
        return model

    def summary(self):
        return self.Generator.summary()

    def save_model(self):
        plot_model(self.Generator.model, to_file='Generator_Model.png')



In [ ]:
#gan

#!/usr/bin/env python3
import sys
import numpy as np
from keras.models import Sequential, Model
# from keras.optimizers import Adam
# from keras.utils import plot_model

class GAN(object):
    def __init__(self,discriminator,generator):
        self.OPTIMIZER = Adam(lr=0.008, beta_1=0.5)
        
        self.Generator = generator
        self.Discriminator = discriminator
        self.Discriminator.trainable = True
        
        self.gan_model = self.model()
        self.gan_model.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)
        # self.save_model()
        self.summary()

    def model(self):
        model = Sequential()
        model.add(self.Generator)
        model.add(self.Discriminator)
        return model

    def summary(self):
        return self.gan_model.summary()

    def save_model(self):
        plot_model(self.gan_model.model, to_file='GAN_Model.png')


In [ ]:
#train

#!/usr/bin/env python3
from keras.datasets import mnist
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import h5py    
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
# from voxelgrid import VoxelGrid


class Trainer:
    def __init__(self, side=16, latent_size=32, epochs =100, batch=32, checkpoint=50, data_dir = ''):
        self.SIDE=side
        self.EPOCHS = epochs
        self.BATCH = batch
        self.CHECKPOINT = checkpoint

        self.load_3D_MNIST(data_dir)
        self.load_2D_encoded_MNIST()
        self.LATENT_SPACE_SIZE = latent_size
        self.LABELS = [1]

        self.generator = Generator(latent_size=self.LATENT_SPACE_SIZE)
        self.discriminator = Discriminator(side=self.SIDE)
        self.gan = GAN(generator=self.generator.Generator, discriminator=self.discriminator.Discriminator)
        
    # Translate data to color
    def array_to_color(self,array, cmap="Oranges"):
        s_m = plt.cm.ScalarMappable(cmap=cmap)
        return s_m.to_rgba(array)[:,:-1]

    def translate(self,x):
        xx = np.ndarray((x.shape[0], 4096, 3))
        for i in range(x.shape[0]):
            xx[i] = self.array_to_color(x[i])
        del x
        return xx
    def load_3D_MNIST(self,input_dir):
        raw = h5py.File(input_dir, 'r')


        self.X_train_3D = np.array(raw['X_train'])
        self.X_train_3D = ( np.float32(self.X_train_3D) - 127.5) / 127.5
        self.X_train_3D = self.translate(self.X_train_3D).reshape(-1, 16, 16, 16, 3)
        self.X_test_3D = np.array(raw['X_test'])
        self.X_test_3D = ( np.float32(self.X_test_3D) - 127.5) / 127.5
        self.X_test_3D = self.translate(self.X_test_3D).reshape(-1, 16, 16, 16, 3)
        self.Y_train_3D = np.array(raw['y_train'])
        self.Y_test_3D = np.array(raw['y_test'])

        return 

    def load_2D_encoded_MNIST(self):
        (_, self.Y_train_2D), (_, self.Y_test_2D) = mnist.load_data()
        self.X_train_2D_encoded = np.load('x_train_encoded.npy')
        self.X_test_2D_encoded = np.load('x_test_encoded.npy')
        return

    def train(self):
        
        count_generated_images = int(self.BATCH/2)
        count_real_images = int(self.BATCH/2)
        for e in range(self.EPOCHS):
            for label in self.LABELS:

                # Grab the Real 3D Samples
                all_3D_samples = self.X_train_3D[np.where(self.Y_train_3D==label)]
                starting_index = randint(0, (len(all_3D_samples)-count_real_images))
                real_3D_samples = all_3D_samples[ starting_index : int((starting_index + count_real_images)) ]
                y_real_labels =  np.ones([count_generated_images,1])

                # Grab Generated Images for this training batch
                all_encoded_samples = self.X_train_2D_encoded[np.where(self.Y_train_2D==label)]
                starting_index = randint(0, (len(all_encoded_samples)-count_generated_images))
                batch_encoded_samples = all_encoded_samples[ starting_index : int((starting_index + count_generated_images)) ]
                batch_encoded_samples = batch_encoded_samples.reshape( count_generated_images, 1, 1, 1,self.LATENT_SPACE_SIZE)

                x_generated_3D_samples = self.generator.Generator.predict(batch_encoded_samples)
                y_generated_labels = np.zeros([count_generated_images,1])

                # Combine to train on the discriminator
                x_batch = np.concatenate( [real_3D_samples, x_generated_3D_samples] )
                y_batch = np.concatenate( [y_real_labels, y_generated_labels] )

                # Now, train the discriminator with this batch
                self.discriminator.Discriminator.trainable = False
                discriminator_loss = self.discriminator.Discriminator.train_on_batch(x_batch,y_batch)[0]
                self.discriminator.Discriminator.trainable = True

                # Generate Noise
                starting_index = randint(0, (len(all_encoded_samples)-self.BATCH))
                x_batch_encoded_samples = all_encoded_samples[ starting_index : int((starting_index + self.BATCH)) ]
                x_batch_encoded_samples = x_batch_encoded_samples.reshape( int(self.BATCH), 1, 1, 1,self.LATENT_SPACE_SIZE)
                y_generated_labels = np.ones([self.BATCH,1])
                generator_loss = self.gan.gan_model.train_on_batch(x_batch_encoded_samples,y_generated_labels)
                print ('Epoch: '+str(int(e))+' Label: '+str(int(label))+', [Discriminator :: Loss: '+str(discriminator_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                if e % self.CHECKPOINT == 0 and e != 0 :
                    self.plot_checkpoint(e,label)
            
        return

    def plot_checkpoint(self,e,label):
        filename = "/content/out/epoch_"+str(e)+"_label_"+str(label)+".png"

        all_encoded_samples = self.X_test_2D_encoded[np.where(self.Y_test_2D==label)]
        index = randint(0, (len(all_encoded_samples)-1))
        batch_encoded_samples = all_encoded_samples[ index ]
        batch_encoded_samples = batch_encoded_samples.reshape( 1, 1, 1, 1,self.LATENT_SPACE_SIZE)

        images = self.generator.Generator.predict(batch_encoded_samples)
        xs = []
        ys = []
        zs = []
        cs = []
        for i in range(16):
            for j in range(16):
                for k in range(16):
                    color = images[0][i][j][k]
                    if np.mean(color)<0.75 and np.mean(color)>0.25:
                        xs.append(i)
                        ys.append(j)
                        zs.append(k)
                        cs.append(color)

        fig = plt.figure()
        ax = fig.gca(projection='3d')
        ax.scatter(xs,ys,zs,alpha=0.1,c=cs)
        plt.savefig(filename)

        return

In [ ]:
# Command Line Argument Method
CUBE_SIDE=16
EPOCHS = 100000
BATCH = 64
CHECKPOINT = 10
LATENT_SPACE_SIZE = 256
DATA_DIR = "/content/3d-mnist/full_dataset_vectors.h5"

trainer = Trainer(side=CUBE_SIDE, \
                 latent_size=LATENT_SPACE_SIZE, \
                 epochs =EPOCHS,\
                 batch=BATCH,\
                 checkpoint=CHECKPOINT,\
                 data_dir = DATA_DIR)
trainer.train()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1, 1, 1, 256)]    0         
                                                                 
 conv3d_transpose_3 (Conv3DT  (None, 4, 4, 4, 256)     4194560   
 ranspose)                                                       
                                                                 
 batch_normalization_8 (Batc  (None, 4, 4, 4, 256)     1024      
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 4, 4, 4, 256)      0         
                                                                 
 conv3d_transpose_4 (Conv3DT  (None, 8, 8, 8, 128)     2097280   
 ranspose)                                                       
                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                 
 conv3d_6 (Conv3D)           (None, 16, 16, 16, 16)    3472      
                                                                 
 batch_normalization_12 (Bat  (None, 16, 16, 16, 16)   64        
 chNormalization)                                                
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 16, 16, 16, 16)    0         
                                                                 
 conv3d_7 (Conv3D)           (None, 16, 16, 16, 32)    13856     
                                                                 
 batch_normalization_13 (Bat  (None, 16, 16, 16, 32)   128       
 chNormalization)                                                
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 16, 16, 16, 32)    0         
                                                                 
 conv3d_8 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:128: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Epoch: 211 Label: 1, [Discriminator :: Loss: 6.478871822357178], [ Generator :: Loss: 1.1065010312449886e-06]
Epoch: 212 Label: 1, [Discriminator :: Loss: 6.216343879699707], [ Generator :: Loss: 1.1179904504388105e-06]
Epoch: 213 Label: 1, [Discriminator :: Loss: 6.395967483520508], [ Generator :: Loss: 1.112848281081824e-06]
Epoch: 214 Label: 1, [Discriminator :: Loss: 6.310398101806641], [ Generator :: Loss: 1.0910425771726295e-06]
Epoch: 215 Label: 1, [Discriminator :: Loss: 6.450102806091309], [ Generator :: Loss: 1.0531235830057994e-06]
Epoch: 216 Label: 1, [Discriminator :: Loss: 6.2627482414245605], [ Generator :: Loss: 1.2085782827853109e-06]
Epoch: 217 Label: 1, [Discriminator :: Loss: 6.424884796142578], [ Generator :: Loss: 1.0435090871396824e-06]
Epoch: 218 Label: 1, [Discriminator :: Loss: 6.340157985687256], [ Generator :: Loss: 1.0456370773681556e-06]
Epoch: 219 Label: 1, [Discriminator :: Loss: 6.198159694671631], [ Generator :: Loss: 1.1211326409465983e-06]
Epoch: 220